<a href="https://colab.research.google.com/github/satishgc227/Derby-project/blob/main/Deep_learning_hyper_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score


!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [33]:
# Load in the data
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv('/content/edited complete.csv',encoding='latin',nrows=10000)

df['race_type\xa0'].unique()
df['course_type'].unique()
df['track_condition\xa0'].unique()
df['track_id'].unique()

s=df[['race_type\xa0','course_type','track_condition\xa0','track_id']]

ohe=OneHotEncoder()
feature_array=ohe.fit_transform(df[['race_type\xa0','course_type','track_condition\xa0','track_id']]).toarray()

feature_labels=ohe.categories_
print(feature_labels)

np.hstack(feature_labels)

feature_labels=np.hstack(feature_labels)
f=pd.DataFrame(feature_array,columns=feature_labels)
pd.concat([df,f],axis=1)

df.drop(['race_date','track_id','course_type','track_condition\xa0','race_type\xa0','jockey\xa0','program_number\xa0','latitude\xa0','longitude\xa0'],axis=1,inplace=True)
df=pd.concat([df,f],axis=1)

df.isnull().any()



"""Removing the null values"""

df.replace([np.inf,-np.inf],np.nan ,inplace=True)
df.dropna(inplace=True)
df.isnull().any()

!pip install movecolumn
import movecolumn as mc
mc.MoveToLast(df,'position_at_finish\xa0')

X = df.iloc[:,:-1]
y = df.iloc[:, -1]




[array(['ALW', 'AOC', 'CLM', 'SST'], dtype=object), array(['D', 'I', 'M', 'O', 'T'], dtype=object), array(['FM ', 'FT ', 'GD ', 'MY ', 'SY '], dtype=object), array(['AQU', 'BEL', 'SAR'], dtype=object)]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.corr()['position_at_finish\xa0']

In [34]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)
scaler = StandardScaler()
scaler.fit(X_train)

# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score


# Set paramaters
params_nn ={
    'neurons': (10, 12),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 100),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=1000, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
| 1         | nan       | 5.51      | 183.1     | 54.88     | 0.7716    | 10.59     | 1.044     |
| 2         | nan       | 0.2023    | 158.0     | 39.09     | 0.3443    | 11.98     | 1.664     |
| 3         | nan       | 0.7307    | 133.0     | 69.7      | 0.2815    | 10.93     | 0.8286    |
| 4         | nan       | 0.6656    | 109.9     | 83.52     | 0.8422    | 11.63     | 6.937     |
| 5         | nan       | 5.195     | 118.6     | 53.71     | 0.03717   | 10.91     | 0.7373    |
| 6         | nan       | 7.355     | 130.2     | 65.22     | 0.2815    | 12.0      | 0.9663    |
| 7         | nan       | 5.539     | 151.5     | 52.4      | 0.7306    | 10.65     | 2.804     |
| 8         | nan       | 2.871     | 105.3     | 93.5      | 0.8157    | 10.07     | 6.604     |
| 9         | nan   

ValueError: ignored

In [ ]:
df.replace([np.inf,-np.inf],np.nan ,inplace=True)
df.dropna(inplace=True)
df.isnull().any()

In [ ]:
# Set paramaters
#params_nn ={
 #   'neurons': (10, 100),
 #   'activation':(0, 9),
  #  'optimizer':(0,7),
   # 'learning_rate':(0.01, 1),
    #'batch_size':(200, 1000),
   # 'epochs':(20, 100)
#}
# Run Bayesian Optimization
#nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
#nn_bo.maximize(init_points=25, n_iter=4)

In [35]:
#Here are the best hyperparameters.

params_nn_ = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_

{'activation': 'elu',
 'batch_size': 183.09302456543637,
 'epochs': 54.884721549693616,
 'learning_rate': 0.7715698477978916,
 'neurons': 10.590650608805753,
 'optimizer': 1.0441406999764284}

In [ ]:
#The following code creates a function for tuning the Neural Network hyperparameters and layers.

In [36]:

df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Create function
def nn_cl_bo2(neurons, activation, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        if normalization > 0.5:
            nn.add(BatchNormalization())
        for i in range(layers1):
            nn.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

#The following code searches for the optimum hyperparameters and layers for the Neural Network model.

params_nn2 ={
    'neurons': (10,20),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}

df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=100, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | nan       | 5.51      | 335.3     | 0.4361    | 0.2308    | 43.63     | 1.298     | 1.045     | 0.426     | 12.39     | 0.3377    | 6.935     |
| 2         | nan       | 2.14      | 265.0     | 0.6696    | 0.1864    | 41.94     | 1.932     | 1.237     | 0.08322   | 19.01     | 0.794     | 5.884     |
| 3         | nan       | 7.337     | 992.8     | 0.5773    | 0.2441    | 53.71     | 1.055     | 1.908     | 0.1143    | 18.17     | 0.6977    | 3.957     |
| 4         | nan       | 2.468     | 998.8     | 0.138     | 0.1846    | 58.8      | 1.81      | 2.456     | 0.3296    | 14.01     | 0.319     | 6.631     |
| 5         | nan       | 8.268     | 851.1     | 0.

ValueError: ignored

In [37]:
#Here are the tuned hyperparameters and layers.
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons'] = round(params_nn_['neurons'])
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
             'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
             'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
             'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]
params_nn_

{'activation': 'elu',
 'batch_size': 335,
 'dropout': 0.4360590193711702,
 'dropout_rate': 0.23077874175693686,
 'epochs': 44,
 'layers1': 1,
 'layers2': 1,
 'learning_rate': 0.42602224734191213,
 'neurons': 12,
 'normalization': 0.33765619188879237,
 'optimizer': <keras.optimizers.optimizer_v2.ftrl.Ftrl at 0x7f49ac24cd50>}

In [38]:
#Fitting Neural Network
def nn_cl_fun():
  nn = Sequential()
  nn.add(Dense(params_nn_['neurons'], input_dim=25, activation=params_nn_['activation']))
  if params_nn_['normalization'] > 0.5:
    nn.add(BatchNormalization())
  for i in range(params_nn_['layers1']):
    nn.add(Dense(params_nn_['neurons'], activation=params_nn_['activation']))
  if params_nn_['dropout'] > 0.5:
    nn.add(Dropout(params_nn_['dropout_rate'], seed=123))
  for i in range(params_nn_['layers2']):
    nn.add(Dense(params_nn_['neurons'], activation=params_nn_['activation']))
  nn.add(Dense(1, activation='sigmoid'))
  nn.compile(loss='binary_crossentropy', optimizer=params_nn_['optimizer'], metrics=['accuracy'])
  return nn
es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
nn = KerasClassifier(build_fn=nn_cl_fun, epochs=params_nn_['epochs'], batch_size=params_nn_['batch_size'],
                         verbose=0)
nn.fit(X_train, y_train, validation_data=(X_test, y_test), verbose=1)

Epoch 1/44
24/24 [==============================] - 1s 12ms/step - loss: 9331.5820 - accuracy: 0.0891 - val_loss: 8900.6953 - val_accuracy: 0.0000e+00
Epoch 2/44
24/24 [==============================] - 0s 4ms/step - loss: 4516.4375 - accuracy: 0.0891 - val_loss: 4193.2842 - val_accuracy: 0.0000e+00
Epoch 3/44
24/24 [==============================] - 0s 4ms/step - loss: 1999.3755 - accuracy: 0.0891 - val_loss: 1829.1677 - val_accuracy: 0.0000e+00
Epoch 4/44
24/24 [==============================] - 0s 4ms/step - loss: 778.3040 - accuracy: 0.0891 - val_loss: 362.2358 - val_accuracy: 0.0000e+00
Epoch 5/44
24/24 [==============================] - 0s 4ms/step - loss: -58.1423 - accuracy: 0.2192 - val_loss: -491.8879 - val_accuracy: 0.0880
Epoch 6/44
24/24 [==============================] - 0s 4ms/step - loss: -488.9399 - accuracy: 0.2716 - val_loss: -1117.3877 - val_accuracy: 0.0880
Epoch 7/44
24/24 [==============================] - 0s 4ms/step - loss: -865.8948 - accuracy: 0.2716 - val_lo

In [39]:
print('X_train dimension= ', X_train.shape)
print('X_test dimension= ', X_test.shape)
print('y_train dimension= ', y_train.shape)
print('y_test dimension= ', y_test.shape)

X_train dimension=  (8000, 25)
X_test dimension=  (2000, 25)
y_train dimension=  (8000,)
y_test dimension=  (2000,)
